In [1]:
import os
import json
import numpy as np
import pandas as pd
import codecs
import spacy
import random

from spacy.training.example import Example

In [2]:
positive_file = "/data/processed/kaggle/coleridge_initiative/processing_1/pos.json"
negative_file = "/data/processed/kaggle/coleridge_initiative/processing_1/neg.json"

In [4]:
def find_indices(sentence, label):
    start_indices = []
    end_indices = []
    start = 0
    while start < len(sentence):
        temp_index = sentence.find(label, start, len(sentence))
        if temp_index == -1:
            start = len(sentence)
        else:
            start = temp_index + len(label)
            start_indices.append(temp_index)
            end_indices.append(temp_index+len(label))
    return start_indices, end_indices

pos_data = json.load(codecs.open(positive_file, "r"))

spacy_data = []
for e_loc, ent in enumerate(pos_data):
    if e_loc %10000 ==0:
        print(e_loc, len(pos_data))
    e_sent = ent["sentence"]
    e_label = ent["label"]
    
    e_id = "%s_%s_%s"%(ent["file"], ent["section"], ent["sentence_id"])
    
    start_indices, end_indices = find_indices(e_sent, e_label)
    
    pos_data[e_loc]["start_indices"] = start_indices
    pos_data[e_loc]["end_indices"] = end_indices
    
    entities = [(s, e, "data") for s,e in zip(start_indices, end_indices)]
    
    spacy_data.append( (e_id, e_sent, {"entities": entities} ) )


0 37667
10000 37667
20000 37667
30000 37667


In [7]:
# verifying entities
for e_loc, ent in enumerate(pos_data):
#     print(e_loc)
    e_label = ent["label"]
    
    for s, e in zip(ent["start_indices"], ent["end_indices"]):
        se_label = ent["sentence"][s:e]
        if e_label != se_label:
            print(1, ent["sentence"])
            print(2, e_label)
            print(3, se_label)
            print(4, s, e)

In [14]:
for tid, text, entities in spacy_data:
    for ent in entities["entities"]:
        if len(text) < ent[1]:
            print(len(text), ent[0], ent[1])

In [ ]:
def train_spacy(data, iterations):
    
    '''
    Arguments
    data: data in spacy format [(id, text, {entities:[(start, end, type)]})]
    iterations: int no of iterations to train
    '''
    TRAIN_DATA = data
    nlp = spacy.blank('en') 
    
    if 'ner' not in nlp.pipe_names:
#         ner = nlp.create_pipe('ner')
        nlp.add_pipe("ner", last=True)
       

    # add labels
#     for _, _, annotations in TRAIN_DATA:
#          for ent in annotations.get('entities'):
#             ner.add_label(ent[2])
    
    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    
    with nlp.disable_pipes(*other_pipes):  # only train NER
        optimizer = nlp.begin_training()
        losses = {}
        old_score = 0
        for itn in range(iterations):
            print("Starting Iteration")
            random.shuffle(TRAIN_DATA)
            
            
            for i in range(len(TRAIN_DATA)):
#                 spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)
                
                doc = nlp.make_doc(TRAIN_DATA[i][1])
                example = Example.from_dict(doc, TRAIN_DATA[i][2])
                
                
                nlp.update([example], drop=0.2, sgd=optimizer, losses=losses)
            print("  Iteration: %s,  Losses: %s"%(itn, losses["ner"] -  old_score))
            old_score = losses["ner"] 
    return nlp
nlp_model = train_spacy(spacy_data, iterations = 10)


Starting Iteration


/data/venv/kaggle_coleridge/lib/python3.6/site-packages/spacy/training/iob_utils.py:142: UserWarning: [W030] Some entities could not be aligned in the text "Once the autoencoder was obtained, the encoder was..." with entities "[(93, 97, 'data')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  entities=ent_str[:50] + "..." if len(ent_str) > 50 else ent_str,
/data/venv/kaggle_coleridge/lib/python3.6/site-packages/spacy/training/iob_utils.py:142: UserWarning: [W030] Some entities could not be aligned in the text "To analyze brain imaging data, we used T1-weighted..." with entities "[(70, 74, 'data')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  entities=ent_str[:50] + "..." if len(ent_str) > 50 else ent_str,
/data/venv/kaggle_coleridge/lib/python3.6/site-packages/sp

/data/venv/kaggle_coleridge/lib/python3.6/site-packages/spacy/training/iob_utils.py:142: UserWarning: [W030] Some entities could not be aligned in the text "Similar to the KBASE dataset, we also made three s..." with entities "[(270, 274, 'data')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  entities=ent_str[:50] + "..." if len(ent_str) > 50 else ent_str,
/data/venv/kaggle_coleridge/lib/python3.6/site-packages/spacy/training/iob_utils.py:142: UserWarning: [W030] Some entities could not be aligned in the text "Table 1 provides demographic and clinical informat..." with entities "[(75, 79, 'data'), (86, 90, 'data')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  entities=ent_str[:50] + "..." if len(ent_str) > 50 else ent_str,
/data/venv/kaggle_coleridge/lib/python

/data/venv/kaggle_coleridge/lib/python3.6/site-packages/spacy/training/iob_utils.py:142: UserWarning: [W030] Some entities could not be aligned in the text "As summarized in Table S1 and Table S2 , significa..." with entities "[(110, 114, 'data'), (151, 155, 'data')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  entities=ent_str[:50] + "..." if len(ent_str) > 50 else ent_str,
/data/venv/kaggle_coleridge/lib/python3.6/site-packages/spacy/training/iob_utils.py:142: UserWarning: [W030] Some entities could not be aligned in the text "This dataset consists of T1w and FLAIR scans of 46..." with entities "[(65, 69, 'data')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  entities=ent_str[:50] + "..." if len(ent_str) > 50 else ent_str,
/data/venv/kaggle_coleridge/lib/pyth

/data/venv/kaggle_coleridge/lib/python3.6/site-packages/spacy/training/iob_utils.py:142: UserWarning: [W030] Some entities could not be aligned in the text "Subjects originally recruited for ADNI-1 and ADNI-..." with entities "[(34, 38, 'data'), (45, 49, 'data'), (101, 105, 'd...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  entities=ent_str[:50] + "..." if len(ent_str) > 50 else ent_str,
/data/venv/kaggle_coleridge/lib/python3.6/site-packages/spacy/training/iob_utils.py:142: UserWarning: [W030] Some entities could not be aligned in the text "The 113 AD, 351 MCI, and 122 CN subjects from the ..." with entities "[(50, 54, 'data'), (83, 87, 'data')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  entities=ent_str[:50] + "..." if len(ent_str) > 50 else ent_str,
/data

...

/data/venv/kaggle_coleridge/lib/python3.6/site-packages/spacy/training/iob_utils.py:142: UserWarning: [W030] Some entities could not be aligned in the text "After all quality checks which include both system..." with entities "[(107, 111, 'data'), (141, 145, 'data')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  entities=ent_str[:50] + "..." if len(ent_str) > 50 else ent_str,
/data/venv/kaggle_coleridge/lib/python3.6/site-packages/spacy/training/iob_utils.py:142: UserWarning: [W030] Some entities could not be aligned in the text "This longitudinal study will use standardized ADNI..." with entities "[(46, 50, 'data'), (176, 180, 'data'), (222, 226, ...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  entities=ent_str[:50] + "..." if len(ent_str) > 50 else ent_str,
/

/data/venv/kaggle_coleridge/lib/python3.6/site-packages/spacy/training/iob_utils.py:142: UserWarning: [W030] Some entities could not be aligned in the text "ADNI inclusion and exclusion criteria are detailed..." with entities "[(0, 4, 'data'), (103, 107, 'data')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  entities=ent_str[:50] + "..." if len(ent_str) > 50 else ent_str,
/data/venv/kaggle_coleridge/lib/python3.6/site-packages/spacy/training/iob_utils.py:142: UserWarning: [W030] Some entities could not be aligned in the text "Thus, the primary goal of the current study is to ..." with entities "[(351, 355, 'data')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  entities=ent_str[:50] + "..." if len(ent_str) > 50 else ent_str,
/data/venv/kaggle_coleridge/lib/python

/data/venv/kaggle_coleridge/lib/python3.6/site-packages/spacy/training/iob_utils.py:142: UserWarning: [W030] Some entities could not be aligned in the text "Images for the current study included original unc..." with entities "[(65, 69, 'data'), (83, 87, 'data'), (91, 95, 'dat...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  entities=ent_str[:50] + "..." if len(ent_str) > 50 else ent_str,
/data/venv/kaggle_coleridge/lib/python3.6/site-packages/spacy/training/iob_utils.py:142: UserWarning: [W030] Some entities could not be aligned in the text "We evaluated the model using data from the Alzheim..." with entities "[(90, 94, 'data')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  entities=ent_str[:50] + "..." if len(ent_str) > 50 else ent_str,
/data/venv/kaggle_coler

/data/venv/kaggle_coleridge/lib/python3.6/site-packages/spacy/training/iob_utils.py:142: UserWarning: [W030] Some entities could not be aligned in the text "Partnerships with the Alzheimer's Association, oth..." with entities "[(81, 85, 'data'), (192, 196, 'data')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  entities=ent_str[:50] + "..." if len(ent_str) > 50 else ent_str,
/data/venv/kaggle_coleridge/lib/python3.6/site-packages/spacy/training/iob_utils.py:142: UserWarning: [W030] Some entities could not be aligned in the text "To this purpose we considered the ADNI2 population..." with entities "[(34, 38, 'data')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  entities=ent_str[:50] + "..." if len(ent_str) > 50 else ent_str,
/data/venv/kaggle_coleridge/lib/python

  Iteration: 0,  Losses: 13656.257560671153
Starting Iteration
